In [1]:
import os

from sionna.mapping import Constellation, Mapper, Demapper
from sionna.fec.ldpc import LDPC5GEncoder, LDPC5GDecoder
from sionna.utils import ebnodb2no
from sionna.channel import AWGN, FlatFadingChannel

import tensorflow as tf
from PIL import Image
import math
import numpy as np

from utils.image import imBatchtoImage

2023-05-24 11:29:09.174389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from utils.datasets import dataset_generator
test_ds = dataset_generator('/dataset/CIFAR10/test/')

Found 10000 files belonging to 10 classes.


In [3]:
'''
heic_file() {

     file_magic                                                  u(32)

     pixel_format                                                u(3)
     alpha1_flag                                                 u(1)
     bit_depth_minus_8                                           u(4)

     color_space                                                 u(4)
     extension_present_flag                                      u(1)
     alpha2_flag                                                 u(1)
     limited_range_flag                                          u(1)
     animation_flag                                              u(1)
     
     picture_width                                               ue7(32)
     picture_height                                              ue7(32)
     
     picture_data_length                                         ue7(32)
     if (extension_present_flag)  
         extension_data_length                                   ue7(32)
         extension_data()
     }

     hevc_header_and_data()
}

Header: maximum 22 Bytes
'''

'\nheic_file() {\n\n     file_magic                                                  u(32)\n\n     pixel_format                                                u(3)\n     alpha1_flag                                                 u(1)\n     bit_depth_minus_8                                           u(4)\n\n     color_space                                                 u(4)\n     extension_present_flag                                      u(1)\n     alpha2_flag                                                 u(1)\n     limited_range_flag                                          u(1)\n     animation_flag                                              u(1)\n     \n     picture_width                                               ue7(32)\n     picture_height                                              ue7(32)\n     \n     picture_data_length                                         ue7(32)\n     if (extension_present_flag)  \n         extension_data_length                                  

In [4]:
class BPGEncoder():
    def __init__(self, working_directory='./analysis/temp'):
        '''
        working_directory: directory to save temp files
                           do not include '/' in the end
        '''
        self.working_directory = working_directory
    
    def run_bpgenc(self, qp, input_dir, output_dir='temp.bpg'):
        if os.path.exists(output_dir):
            os.remove(output_dir)
        os.system(f'bpgenc {input_dir} -q {qp} -o {output_dir} -f 444')

        if os.path.exists(output_dir):
            return os.path.getsize(output_dir)
        else:
            return -1
    
    def get_qp(self, input_dir, byte_threshold, output_dir='temp.bpg'):
        '''
        iteratively finds quality parameter that maximizes quality given the byte_threshold constraint
        '''
        # rate-match algorithm
        quality_max = 51
        quality_min = 0
        quality = (quality_max - quality_min) // 2
        
        while True:
            qp = 51 - quality
            bytes = self.run_bpgenc(qp, input_dir, output_dir)
            if quality == 0 or quality == quality_min or quality == quality_max:
                break
            elif bytes > byte_threshold and quality_min != quality - 1:
                quality_max = quality
                quality -= (quality - quality_min) // 2
            elif bytes > byte_threshold and quality_min == quality - 1:
                quality_max = quality
                quality -= 1
            elif bytes < byte_threshold and quality_max > quality:
                quality_min = quality
                quality += (quality_max - quality) // 2
            else:
                break
        
        return qp
    
    def encode(self, image_array, max_bytes, header_bytes=22):
        '''
        image_array: uint8 numpy array with shape (b, h, w, c)
        max_bytes: int, maximum bytes of the encoded image file (exlcuding header bytes)
        header_bytes: the size of BPG header bytes (to be excluded in image file size calculation)
        '''

        input_dir = f'{self.working_directory}/temp_enc.png'
        output_dir = f'{self.working_directory}/temp_enc.bpg'

        im = Image.fromarray(image_array, 'RGB')
        im.save(input_dir)

        qp = self.get_qp(input_dir, max_bytes + header_bytes, output_dir)
        
        if self.run_bpgenc(qp, input_dir, output_dir) < 0:
            raise RuntimeError("BPG encoding failed")

        # read binary and convert it to numpy binary array with float dtype
        return np.unpackbits(np.fromfile(output_dir, dtype=np.uint8)).astype(np.float32)


In [5]:
class LDPCTransmitter():
    '''
    Transmits given bits (float array of '0' and '1') with LDPC.
    '''
    def __init__(self, k, n, m, esno_db, channel='AWGN'):
        '''
        k: data bits per codeword (in LDPC)
        n: total codeword bits (in LDPC)
        m: modulation order (in m-QAM)
        esno_db: channel SNR
        channel: 'AWGN' or 'Rayleigh'
        '''
        self.k = k
        self.n = n
        self.num_bits_per_symbol = round(math.log2(m))

        constellation_type = 'qam' if m != 2 else 'pam'
        self.constellation = Constellation(constellation_type, num_bits_per_symbol=self.num_bits_per_symbol)
        self.mapper = Mapper(constellation=self.constellation)
        self.demapper = Demapper('app', constellation=self.constellation)
        self.channel = AWGN() if channel == 'AWGN' else FlatFadingChannel
        self.encoder = LDPC5GEncoder(k=self.k, n=self.n)
        self.decoder = LDPC5GDecoder(self.encoder, num_iter=20)
        self.esno_db = esno_db
    

    def send(self, source_bits):
        '''
        source_bits: float np array of '0' and '1', whose total # of bits is divisible with k
        '''
        lcm = np.lcm(self.k, self.num_bits_per_symbol)
        source_bits_pad = tf.pad(source_bits, [[0, math.ceil(len(source_bits)/lcm)*lcm - len(source_bits)]])
        u = np.reshape(source_bits_pad, (-1, self.k))

        no = ebnodb2no(self.esno_db, num_bits_per_symbol=1, coderate=1)
        c = self.encoder(u)
        x = self.mapper(c)
        y = self.channel([x, no])
        llr_ch = self.demapper([y, no])
        u_hat = self.decoder(llr_ch)

        return tf.reshape(u_hat, (-1))[:len(source_bits)]

In [6]:
class BPGDecoder():
    def __init__(self, working_directory='./analysis/temp'):
        '''
        working_directory: directory to save temp files
                           do not include '/' in the end
        '''
        self.working_directory = working_directory
    
    def run_bpgdec(self, input_dir, output_dir='temp.png'):
        if os.path.exists(output_dir):
            os.remove(output_dir)
        os.system(f'bpgdec {input_dir} -o {output_dir}')

        if os.path.exists(output_dir):
            return os.path.getsize(output_dir)
        else:
            return -1

    def decode(self, bit_array, image_shape):
        '''
        returns decoded result of given bit_array.
        if bit_array is not decodable, then returns the mean CIFAR-10 pixel values.

        byte_array: float array of '0' and '1'
        image_shape: used to generate image with mean pixel values if the given byte_array is not decodable
        '''
        input_dir = f'{self.working_directory}/temp_dec.bpg'
        output_dir = f'{self.working_directory}/temp_dec.png'

        byte_array = np.packbits(bit_array.astype(np.uint8))
        with open(input_dir, "wb") as binary_file:
            binary_file.write(byte_array.tobytes())

        cifar_mean = np.array([0.4913997551666284, 0.48215855929893703, 0.4465309133731618]) * 255
        cifar_mean = np.reshape(cifar_mean, [1] * (len(image_shape) - 1) + [3]).astype(np.uint8)

        if self.run_bpgdec(input_dir, output_dir) < 0:
            # print('warning: Decode failed. Returning mean pixel value')
            return 0 * np.ones(image_shape) + cifar_mean
        else:
            x = np.array(Image.open(output_dir).convert('RGB'))
            if x.shape != image_shape:
                return 0 * np.ones(image_shape) + cifar_mean
            return x


In [43]:
# BPG + LDPC
from tqdm.auto import tqdm

bpgencoder = BPGEncoder()
bpgdecoder = BPGDecoder()

bw_ratio = [1/12, 1/6, 1/4, 1/3, 1/2]
snrs = [0, 10]
mcs = [(k, n, m) for k, n in [(3072, 6144), (3072, 4608), (1536, 4608)] for m in (2, 4, 16, 64)]
'''
(3072, 6144), (3072, 4608), (1536, 4608)
BPSK, 4-QAM, 16-QAM, 64-QAM
'''

for esno_db in snrs:
    for bw in bw_ratio:
        for k, n, m in mcs:
            i = 0
            psnr = 0
            ssim = 0
            total_images = 0
            ldpctransmitter = LDPCTransmitter(k, n, m, esno_db, 'AWGN')
            for image, _ in tqdm(test_ds):
                b, _, _, _ = image.shape
                image = tf.cast(imBatchtoImage(image), tf.uint8)
                max_bytes = b * 32 * 32 * 3 * bw * math.log2(m) * k / n / 8
                src_bits = bpgencoder.encode(image.numpy(), max_bytes)
                rcv_bits = ldpctransmitter.send(src_bits)
                
                decoded_image = bpgdecoder.decode(rcv_bits.numpy(), image.shape)
                total_images += b
                psnr = (total_images - b) / (total_images) * psnr + float(b * tf.image.psnr(decoded_image, image, max_val=255)) / (total_images)
                ssim = (total_images - b) / (total_images) * ssim + float(b * tf.image.ssim(tf.cast(decoded_image, dtype=tf.float32), tf.cast(image, dtype=tf.float32), max_val=255)) / (total_images)

            print(f'SNR={esno_db},bw={bw},k={k},n={n},m={m},PSNR={psnr:.2f},SSIM={ssim:.2f}')


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=0,bw=0.08333333333333333,k=3072,n=6144,m=2,PSNR=19.58,SSIM=0.59


  0%|          | 0/10 [00:00<?, ?it/s]

Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image


SNR=0,bw=0.08333333333333333,k=3072,n=6144,m=4,PSNR=6.57,SSIM=0.12


...

SNR=10,bw=0.5,k=1536,n=4608,m=4,PSNR=27.97,SSIM=0.92


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=10,bw=0.5,k=1536,n=4608,m=16,PSNR=32.83,SSIM=0.97


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=10,bw=0.5,k=1536,n=4608,m=64,PSNR=36.40,SSIM=0.98


In [25]:
# BPG + Capacity
from tqdm.auto import tqdm

bpgencoder = BPGEncoder()
bpgdecoder = BPGDecoder()

snrs = [0, 10]
mcs = [(1, 1, 1)] # dummy
bw_ratio = [1/12, 1/6, 1/4, 1/3, 1/2]
'''
(3072, 6144), (3072, 4608), (1536, 4608)
BPSK, 4-QAM, 16-QAM, 64-QAM
'''

for esno_db in snrs:
    for bw in bw_ratio:
        for _, _, _ in mcs:
            i = 0
            psnr = 0
            ssim = 0
            total_images = 0
            for image, _ in tqdm(test_ds):
                b, _, _, _ = image.shape
                image = tf.cast(imBatchtoImage(image), tf.uint8)
                max_bytes = b * 32 * 32 * 3 * bw * math.log2(1 + 10 ** (esno_db/10)) / 8
                src_bits = bpgencoder.encode(image.numpy(), max_bytes)
                decoded_image = bpgdecoder.decode(src_bits, image.shape)
                total_images += b
                psnr = (total_images - b) / (total_images) * psnr + float(b * tf.image.psnr(decoded_image, image, max_val=255)) / (total_images)
                ssim = (total_images - b) / (total_images) * ssim + float(b * tf.image.ssim(tf.cast(decoded_image, dtype=tf.float32), tf.cast(image, dtype=tf.float32), max_val=255)) / (total_images)

            print(f'SNR={esno_db},bw={bw},PSNR={psnr:.2f},SSIM={ssim:.2f}')


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=0,PSNR=21.66,SSIM=0.72


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=10,PSNR=26.82,SSIM=0.89


  0%|          | 0/10 [00:00<?, ?it/s]

...

SNR=10,PSNR=37.69,SSIM=0.99


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=0,PSNR=30.49,SSIM=0.95


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=10,PSNR=42.29,SSIM=1.00


In [7]:
# BPG + LDPC (1/6 BW ratio)
from tqdm.auto import tqdm

bpgencoder = BPGEncoder()
bpgdecoder = BPGDecoder()

bw_ratio = [1/6]
snrs = [0, 2, 5, 7, 10, 12, 15]
mcs = [(k, n, m) for k, n in [(3072, 6144), (3072, 4608), (1536, 4608)] for m in (2, 4, 16, 64)]
'''
(3072, 6144), (3072, 4608), (1536, 4608)
BPSK, 4-QAM, 16-QAM, 64-QAM
'''

for esno_db in snrs:
    for bw in bw_ratio:
        for k, n, m in mcs:
            i = 0
            psnr = 0
            ssim = 0
            total_images = 0
            ldpctransmitter = LDPCTransmitter(k, n, m, esno_db, 'AWGN')
            for image, _ in tqdm(test_ds):
                b, _, _, _ = image.shape
                image = tf.cast(imBatchtoImage(image), tf.uint8)
                max_bytes = b * 32 * 32 * 3 * bw * math.log2(m) * k / n / 8
                src_bits = bpgencoder.encode(image.numpy(), max_bytes)
                rcv_bits = ldpctransmitter.send(src_bits)
                
                decoded_image = bpgdecoder.decode(rcv_bits.numpy(), image.shape)
                total_images += b
                psnr = (total_images - b) / (total_images) * psnr + float(b * tf.image.psnr(decoded_image, image, max_val=255)) / (total_images)
                ssim = (total_images - b) / (total_images) * ssim + float(b * tf.image.ssim(tf.cast(decoded_image, dtype=tf.float32), tf.cast(image, dtype=tf.float32), max_val=255)) / (total_images)

            print(f'SNR={esno_db},bw={bw},k={k},n={n},m={m},PSNR={psnr:.2f},SSIM={ssim:.2f}')


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=0,bw=0.16666666666666666,k=3072,n=6144,m=2,PSNR=21.66,SSIM=0.72


  0%|          | 0/10 [00:00<?, ?it/s]

Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image
Could not decode image


SNR=0,bw=0.16666666666666666,k=3072,n=6144,m=4,PSNR=6.57,SSIM=0.12


...

SNR=15,bw=0.16666666666666666,k=1536,n=4608,m=4,PSNR=22.37,SSIM=0.75


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=15,bw=0.16666666666666666,k=1536,n=4608,m=16,PSNR=25.43,SSIM=0.86


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=15,bw=0.16666666666666666,k=1536,n=4608,m=64,PSNR=27.96,SSIM=0.92


In [8]:
# BPG + Capacity
from tqdm.auto import tqdm

bpgencoder = BPGEncoder()
bpgdecoder = BPGDecoder()

bw_ratio = [1/6]
snrs = [0, 2, 5, 7, 10, 12, 15]
mcs = [(1,1,1)] # dummy
'''
(3072, 6144), (3072, 4608), (1536, 4608)
BPSK, 4-QAM, 16-QAM, 64-QAM
'''

for esno_db in snrs:
    for bw in bw_ratio:
        for _, _, _ in mcs:
            i = 0
            psnr = 0
            ssim = 0
            total_images = 0
            for image, _ in tqdm(test_ds):
                b, _, _, _ = image.shape
                image = tf.cast(imBatchtoImage(image), tf.uint8)
                max_bytes = b * 32 * 32 * 3 * bw * math.log2(1 + 10 ** (esno_db/10)) / 8
                src_bits = bpgencoder.encode(image.numpy(), max_bytes)
                decoded_image = bpgdecoder.decode(src_bits, image.shape)
                total_images += b
                psnr = (total_images - b) / (total_images) * psnr + float(b * tf.image.psnr(decoded_image, image, max_val=255)) / (total_images)
                ssim = (total_images - b) / (total_images) * ssim + float(b * tf.image.ssim(tf.cast(decoded_image, dtype=tf.float32), tf.cast(image, dtype=tf.float32), max_val=255)) / (total_images)

            print(f'SNR={esno_db},bw={bw},PSNR={psnr:.2f},SSIM={ssim:.2f}')


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=0,bw=0.16666666666666666,PSNR=24.13,SSIM=0.82


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=2,bw=0.16666666666666666,PSNR=25.65,SSIM=0.87


  0%|          | 0/10 [00:00<?, ?it/s]

...

SNR=10,bw=0.16666666666666666,PSNR=31.59,SSIM=0.96


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=12,bw=0.16666666666666666,PSNR=32.83,SSIM=0.97


  0%|          | 0/10 [00:00<?, ?it/s]

SNR=15,bw=0.16666666666666666,PSNR=34.54,SSIM=0.98
